In [43]:
import tweepy       # python wrapper for twitter api
import json
import time
import requests
# step 0, get your own twitter credentials!
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline
import collections

In [44]:
# this is my own personal twitter api information
# if you could be so kind as to sign up yourself on both twitter and mashape that'd be great :)
# It's FREEEEEEE
api_key = 'g5uPIpw80nULQI1gfklv2zrh4'
api_secret = 'cOWvNWxYvPmEZ0ArZVeeVVvJu41QYHdUS2GpqIKtSQ1isd5PJy'
access_token = '49722956-TWl8J0aAS6KTdcbz3ppZ7NfqZEmrwmbsb9cYPNELG'
access_secret = '3eqrVssF3ppv23qyflyAto8wLEiYRA8sXEPSghuOJWTub'

In [45]:
# Documentation is your friend! http://docs.tweepy.org/en/v3.1.0/
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth) # returns a tweepy authorization handler object


In [46]:
# 1. Use Twitter API to stream and store tweets (300 per candidate)
# explain geocode is doing
bernie_tweets =  api.search(q="#berniesanders", count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#bernie', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#FeelTheBern', count=100, geocode="37.781157,-122.398720,50mi")
hillary_tweets = api.search(q='#ImWithHer', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#hillaryclinton', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#hillary2016', count=100, geocode="37.781157,-122.398720,50mi")
the_donald_tweets = api.search(q='#donaldtrump', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#trump2016', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#trump', count=100, geocode="37.781157,-122.398720,50mi") + api.search(q='#makeamericagreatagain', count=100, geocode="37.781157,-122.398720,50mi")

# geocode is looking for 100 tweets for each canididate in 3 different locations 

In [47]:
the_donald_tweets[0]

Status(contributors=None, truncated=False, text=u'@bwinsf #LiarTrump #DonaldTrump @realDonaldTrump  So Donald, do you lie or just embellish the truth?', is_quote_status=False, in_reply_to_status_id=None, id=731928607266725888L, favorite_count=0, _api=<tweepy.api.API object at 0x000000000B351240>, author=User(follow_request_sent=False, has_extended_profile=True, profile_use_background_image=True, _json={u'follow_request_sent': False, u'has_extended_profile': True, u'profile_use_background_image': True, u'default_profile_image': False, u'id': 209657449, u'profile_background_image_url_https': u'https://abs.twimg.com/images/themes/theme1/bg.png', u'verified': False, u'profile_text_color': u'333333', u'profile_image_url_https': u'https://pbs.twimg.com/profile_images/582629807826956288/ppmgQmDA_normal.jpg', u'profile_sidebar_fill_color': u'DDEEF6', u'entities': {u'description': {u'urls': []}}, u'followers_count': 9, u'profile_sidebar_border_color': u'C0DEED', u'id_str': u'209657449', u'profi

In [48]:
# wrappers come with built in python attributes and methods!
print dir(the_donald_tweets[0])

['__class__', '__delattr__', '__dict__', '__doc__', '__eq__', '__format__', '__getattribute__', '__getstate__', '__hash__', '__init__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', '_json', 'author', 'contributors', 'coordinates', 'created_at', 'destroy', 'entities', 'favorite', 'favorite_count', 'favorited', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'metadata', 'parse', 'parse_list', 'place', 'retweet', 'retweet_count', 'retweeted', 'retweets', 'source', 'source_url', 'text', 'truncated', 'user']


In [49]:
print the_donald_tweets[0].created_at
print the_donald_tweets[0].text
print the_donald_tweets[0].favorite_count
print the_donald_tweets[0].favorited
print the_donald_tweets[0].lang
print the_donald_tweets[0].geo
print the_donald_tweets[0].retweet_count
print the_donald_tweets[0].retweeted
#print the_donald_tweets[0].possibly_sensitive
print the_donald_tweets[0].coordinates



2016-05-15 19:25:53
@bwinsf #LiarTrump #DonaldTrump @realDonaldTrump  So Donald, do you lie or just embellish the truth?
0
False
en
None
0
False
None


In [50]:
def parse_status_objects(tweet_list, candidate, party):
    modified_tweets = list()
    for x in tweet_list:
        modified_tweets.append( [
               x.created_at,
               x.text,
               x.favorite_count,
               x.favorited,
               x.lang,
               x.geo,
               x.retweet_count,
               x.retweeted,
               x.coordinates,
               candidate,
               party
               ])
    return modified_tweets
       
hillary_tweets = parse_status_objects(hillary_tweets, 0, 1)
the_donald_tweets = parse_status_objects(the_donald_tweets, 2, 0)
bernie_tweets = parse_status_objects(bernie_tweets, 1, 1)

In [51]:
print bernie_tweets[0]
print the_donald_tweets[0]
print hillary_tweets[0]

[datetime.datetime(2016, 5, 15, 19, 21, 47), u'Yes I know my vote for #BernieSanders will probably not amount to anything  more than a gesture. #BernieOrBlue', 0, False, u'en', None, 0, False, None, 1, 1]
[datetime.datetime(2016, 5, 15, 19, 25, 53), u'@bwinsf #LiarTrump #DonaldTrump @realDonaldTrump  So Donald, do you lie or just embellish the truth?', 0, False, u'en', None, 0, False, None, 2, 0]
[datetime.datetime(2016, 5, 15, 19, 11, 31), u'.@Reince correctly states that "earthquakes make things happen." So, let\'s elect an earthquake. Stupid man... #Trump2016 #ImWithHer', 0, False, u'en', None, 0, False, None, 0, 1]


In [52]:
# 2 Create a dataframe that combines all of the tweets from each candidate
# To do so, you will need to concat the data frames
# so each row is a tweet and your columns should be 
# date, text, favorite_count,favorited, language, geocode, rwtweet count, retweeted, coordinates

A_tweets = bernie_tweets + the_donald_tweets + hillary_tweets

all_tweets = pd.DataFrame(A_tweets, columns=['date', 'text', 'favorite_count','favorited', 'language', 'geocode', 'retweet count', 'retweeted', 'coordinates', 'candidate', 'party'])

In [53]:
# 3 Create a function to take in a string and output the textblob sentiment of that string

# define a function that accepts text and returns the polarity

def detect_sentiment(text):
    
    return TextBlob(text.encode('ascii', 'ignore')).sentiment.polarity

def convert(text):
    return text.encode('ascii', 'ignore')


In [54]:
# 4 Add a column to your data frame called 'sentiment' which holds the sentiment of that tweet (hint: use the function from #3)

all_tweets['sentiment'] = all_tweets.text.apply(detect_sentiment)
all_tweets['text'] = all_tweets.text.apply(convert)
all_tweets.to_csv('as_tweets.csv')

In [76]:
# 5 create a word count column, which holds the number of words in the text column

# use countvectorizer to create a document-term matrix and concatinate it to your main dataframe
#(hint you should now have thousands of columns)

pd.set_option('display.max_rows',500)
pd.set_option('display.notebook_repr_html',True)
pd.set_option('display.max_columns',200)




vect = CountVectorizer()
doc_matrix= vect.fit_transform(all_tweets['text'])

tf = pd.DataFrame(doc_matrix.toarray(), columns=vect.get_feature_names())

#tf.head()

print "all_tweets" 
print all_tweets.shape

print "tf"
print tf.shape


#concatinate 

#all_df = all_tweets + tf 

#all_df=pd.merge(all_tweets,tf,axis= 1)

all_tweets1= pd.merge(all_tweets, tf , how='left', left_index= True, right_index= True )

print "all_tweets1"
print all_tweets1.shape 

all_tweets1.head()


all_tweets
(991, 12)
tf
(991, 2130)
all_tweets1
(991, 2142)


,date,text,favorite_count,favorited,language,geocode,retweet count,retweeted_x,coordinates,candidate,party_x,sentiment,06ocatdjyd,0b0botchgo,0kbmcdjmym,0lxtpapnyh,0po3kgf9ly,0ukjei1qrl,0uz0de8t2t,0w3amjkm5i,10,100,101,10x,112,12p,12th,13,14,17th,1990s,1dmlxivwxa,1vxwmguxai,20,2000,2016,2016uspresidentialelections,2018,20fwsjl1en,23rd,25,27g2vitm7m,27kmahfubx,2bysl,2ctiflw3fj,2dwo7n9lro,2jrqokt3w5,2kuxtxeq9g,2zhplwyhsc,300,36ygewam2l,38,3f,3ksh5gguup,3lnqa2td6n,3nhy5ucc8p,40,43,4ao3z572vn,4bqce752jr,4kaolpg8z8,4qtjurmkjr,50,579n7kqwob,5bdso7,5bdso7gq2u,5ezfm5l4zf,5hf3uwkn4z,5onukmhwqe,5tgtbquqsq,6fibp6o1l7,6pnnuhvqop,75,76x3ovrnyx,7csswtoias,7favhwnol0,7ht3dodqdi,7wor8lkcds,80,80s,81,87,8d1vu5taow,8e2uq6ysrx,8ekaysggfx,8fukn6nuds,8pzsvcpuef,90,90s,91,99,9aehkamzuw,9dk4ktyfs9,9lt9rcfn69,9n5jzmkp1k,9ttznmheys,9w0diomo70,a07n2psjaw,a1vlvqvff5,aapiforbernie,...,week,weekend,weenie,weigh,welcomes,well,welovebarbaraboxer,weneedhillary,were,westvirginia,westwing,wghnjzmv5p,what,when,whenever,where,which,while,white,who,whos,why,wife,wild,will,win,winedivergirl,winner,winning,wins,with,without,woman,womancard,women,women4trump,womenfortrump,won,wonder,wondering,wont,word,wordpressdotcom,words,work,workers,working,world,worth,worthless,would,write,writers,wrong,wupbxpluuk,wv4trump,xgztf8n3dm,xh4cscv3rk,xhbq1ajtyf,xifn0xzsut,xqg8lpz0pp,xty05qbbw5,xuh6xt9pwg,xupeo24n5f,xvieelbnmi,xvr3yjzx81,xzve55n4gm,y3zyvztqpk,yas,ydpkfhclr8,year,years,yerypuzqrm,yes,yet,yickl0dj7s,ymb4ykvmep,ymp7gvllnd,yntfym56yv,yo,yolibeans,york,you,young,your,youtube,yrs,yse2ibkdz8,yu5cps7lnz,yuh2quagd6,z8fukzyrt4,zbegb208k1,zciefbume7,zdww3ahzdg,zesty_enviro,zesty_lambo,zhwdnoaqub,zqpo3qncj8,zt58jck8ir,zynmvhmvvu
0,2016-05-15 19:21:47,Yes I know my vote for #BernieSanders will pro...,0,False,en,None,0,False,None,1,1,0.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2016-05-15 19:16:01,RT @TheAcaActivist: #BernieSanders https://t.c...,0,False,und,None,1,False,None,1,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2016-05-15 19:10:11,I voted for #BernieSanders &amp; I did it with...,1,False,en,None,0,False,None,1,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2016-05-15 19:01:34,#BernieSanders https://t.co/XVR3YjzX81,1,False,und,None,1,False,None,1,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2016-05-15 18:45:53,.@theEcoNEST get a #CrossOverBallot to vote fo...,0,False,en,None,0,False,None,1,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [81]:
# Create another dataframe to drop catigorical data 

kmean_df= all_tweets1.drop(['date','text','language','geocode','coordinates'],axis=1)




In [83]:
# 6 Use kmean to create clusters for the dataframe
# define X

X= kmean_df

# K-means with 3 clusters
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, random_state=1)
km.fit(X)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=1, tol=0.0001,
    verbose=0)

In [61]:
#check for null 
all_tweets.isnull().sum()

date                0
text                0
favorite_count      0
favorited           0
language            0
geocode           990
retweet count       0
retweeted           0
coordinates       990
candidate           0
party               0
sentiment           0
dtype: int64

In [91]:
# 7 Explore the clusters
# What is interesting? Surprising? Can you draw any inferences about each cluster?
# Write your answer to these questions in markdown



########################## edit this ####################

# review the cluster labels
km.labels_



# save the cluster labels and sort by cluster
all_tweets1['cluster'] = km.labels_
all_tweets1.sort('cluster')


# calculate the mean of each feature for each cluster

print "Mean of each feature for each cluster"

all_tweets1.groupby('cluster').mean()

#cluster count

all_tweets1.cluster.value_counts()


# calculate SC for K=3

from sklearn import metrics
metrics.silhouette_score(X, km.labels_)



# save the DataFrame of cluster centers
#centers = all_tweets.groupby('cluster').mean()

# allow plots to appear in the notebook
#%matplotlib inline
#import matplotlib.pyplot as plt
#plt.rcParams['font.size'] = 14


# create a "colors" array for plotting
#import numpy as np
#colors = np.array(['red', 'green', 'blue', 'yellow'])


# scatter plot of calories versus alcohol, colored by cluster (0=red, 1=green, 2=blue)
#plt.scatter(beer.calories, beer.alcohol, c=colors[list(beer.cluster)], s=50)

# cluster centers, marked by "+"
#plt.scatter(centers.calories, centers.alcohol, linewidths=3, marker='+', s=300, c='black')

# add labels
#plt.xlabel('calories')
#plt.ylabel('alcohol')

# scatter plot matrix (0=red, 1=green, 2=blue)
#pd.scatter_matrix(X, c=colors[list(beer.cluster)], figsize=(10,10), s=100)




C:\Users\sulaiman\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


0    972
1     18
2      1
Name: cluster, dtype: int64

In [ ]:
# 8 Create a column called 'democrat' and assign it to the corresponding candidate name for all the records
# 1 for bernie and hilary and 0 for donald trump



In [ ]:
# 9 Estimate classification models using knn and logistic regession.  


## knn 

# train/test_Split
from sklearn.cross_validation import train_test_split


features= all_tweets1[['favorited','retweet count','sentiment']]

response_knn= all_tweets1['candidate']


    # split data into training set and test set
x_train, x_test, y_train, y_test \
= train_test_split(features,response_kmean, random_state=4)


#  confusion matrix


# logistic regression



# Use a three class response variable for knn.  For logistic regression predict democrat or republic?





#Assess the pros and cons of both approaches






In [ ]:
# 10 Initialize a classification model (for both knn and logistic)





In [ ]:
# 11 Train your classification model  (for both knn and logistic)

In [ ]:
# 12 Predict on your test features and targets (for both knn and logistic)

In [ ]:
# 13 output your accuracy (for both knn and logistic)

In [ ]:
# 14 Create a confusion matrix for your predictions (for both knn and logistic)

In [ ]:
# 15 Who is the easiest candidate to predict? Why do you think? Who is the hardest? Why do you think?
# Write your answers in markdown (for both knn and logistic)

In [ ]:
# 16 What are some extra features (data points) you would like to add to this data set. Explain in markdown.
# (hint maybe something like a boolean, "has_hashtag")

In [ ]:
# 17 Try adding extra features to improve the quality of your predictions (for both knn and logistic)